In [4]:
#include <stdio.h>
#include <stdlib.h>

// Fonction pour allouer une matrice n x m
int** create_matrix(int n, int m) {
    int** mat = (int**)malloc(n * sizeof(int*));
    for (int i = 0; i < n; i++) {
        mat[i] = (int*)calloc(m, sizeof(int));
    }
    return mat;
}

// Fonction pour libérer une matrice
void free_matrix(int** mat, int n) {
    for (int i = 0; i < n; i++) {
        free(mat[i]);
    }
    free(mat);
}

// Fonction pour afficher une matrice
void print_matrix(int** mat, int n, int m) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < m; j++) {
            printf("%d ", mat[i][j]);
        }
        printf("\n");
    }
}

// Fonction pour échanger deux lignes d'une matrice
void swap_rows(int** mat, int row1, int row2, int cols) {
    for (int j = 0; j < cols; j++) {
        int temp = mat[row1][j];
        mat[row1][j] = mat[row2][j];
        mat[row2][j] = temp;
    }
}

// Fonction principale d'inversion
int** invert_binary_matrix(int** A, int n) {
    // Créer la matrice augmentée [A|I]
    int** M = create_matrix(n, 2*n);
    
    // Remplir la partie gauche (A)
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            M[i][j] = A[i][j];
        }
    }
    
    // Remplir la partie droite (I)
    for (int i = 0; i < n; i++) {
        M[i][n+i] = 1; //Matrice identité
    }

    // Pivot de Gauss-Jordan
    for (int col = 0; col < n; col++) {
        // Trouver le pivot
        int pivot = -1;
        for (int row = col; row < n; row++) {
            if (M[row][col] == 1) {
                pivot = row;
                break;
            }
        }
        
        if (pivot == -1) {
            printf("Matrice non inversible (colonne %d sans pivot)\n", col+1);
            free_matrix(M, n);
            return NULL;
        }
        
        // Échanger les lignes si nécessaire
        if (pivot != col) {
            swap_rows(M, col, pivot, 2*n);
        }
        
        // Élimination
        for (int row = 0; row < n; row++) {
            if (row != col && M[row][col] == 1) {
                for (int j = 0; j < 2*n; j++) {
                    M[row][j] ^= M[col][j]; // Addition GF(2)
                }
            }
        }
    }
    
    // Extraire l'inverse
    int** A_inv = create_matrix(n, n);
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            A_inv[i][j] = M[i][n+j];
        }
    }
    
    free_matrix(M, n);
    return A_inv;
}

// Fonction de test
void test_inversion() {
    // Définir nos matrices de test
    int* test_matrices[] = {
        (int[]){1,0,0,1}, // Matrice 2x2
        (int[]){1,1,0,1},  // Matrice 2x2
        (int[]){1,0,0,1,0,1,0,1,1}, // Matrice 3x3
        (int[]){1,0,0,0,0,1,0,1,1,1,1,1,0,0,1,1} // Matrice 4x4
    };
    int sizes[] = {2, 2, 3, 4};
    int num_tests = 4;
    
    for (int t = 0; t < num_tests; t++) {
        int n = sizes[t];
        int** A = create_matrix(n, n);
        
        // Copier les données du test
        for (int i = 0; i < n; i++) {
            for (int j = 0; j < n; j++) {
                A[i][j] = test_matrices[t][i*n + j];
            }
        }
        
        printf("\n=== Test %d ===\nMatrice originale A:\n", t+1);
        print_matrix(A, n, n);
        
        int** A_inv = invert_binary_matrix(A, n);
        
        if (A_inv != NULL) {
            printf("\nMatrice inverse A_inv:\n");
            print_matrix(A_inv, n, n);
            
            // Vérification
            printf("\nVerification de A*A_inv (doit etre l'identite):\n");
            int** product = create_matrix(n, n);
            for (int i = 0; i < n; i++) {
                for (int j = 0; j < n; j++) {
                    product[i][j] = 0;
                    for (int k = 0; k < n; k++) {
                        product[i][j] ^= (A[i][k] & A_inv[k][j]);
                    }
                }
            }
            print_matrix(product, n, n);
            
            free_matrix(product, n);
            free_matrix(A_inv, n);
        }
        
        free_matrix(A, n);
    }
}

int main() {
    test_inversion();
    return 0;
}


=== Test 1 ===
Matrice originale A:
1 0 
0 1 

Matrice inverse A_inv:
1 0 
0 1 

Verification de A*A_inv (doit etre l'identite):
1 0 
0 1 

=== Test 2 ===
Matrice originale A:
1 1 
0 1 

Matrice inverse A_inv:
1 1 
0 1 

Verification de A*A_inv (doit etre l'identite):
1 0 
0 1 

=== Test 3 ===
Matrice originale A:
1 0 0 
1 0 1 
0 1 1 

Matrice inverse A_inv:
1 0 0 
1 1 1 
1 1 0 

Verification de A*A_inv (doit etre l'identite):
1 0 0 
0 1 0 
0 0 1 

=== Test 4 ===
Matrice originale A:
1 0 0 0 
0 1 0 1 
1 1 1 1 
0 0 1 1 

Matrice inverse A_inv:
1 0 0 0 
1 0 1 1 
1 1 1 0 
1 1 1 1 

Verification de A*A_inv (doit etre l'identite):
1 0 0 0 
0 1 0 0 
0 0 1 0 
0 0 0 1 
